In [926]:
## Import required packages
## Base modules
import random
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Encoding Modules
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [3]:
# Check local GPU is running, uncomment and run if using a local GPU
#torch.cuda.current_device()
#torch.cuda.device(0)
#torch.cuda.device_count()
#torch.cuda.get_device_name(0)

In [937]:
## We'll build a helper function to generate simulated reads.
## Could've gone for specifying probabilities per base,
## but as it's a field test we're keeping it simple.

def random_dna_sequence(length):
    return np.array(list(''.join(random.choice('ACTG') for _ in range(length))))

# Function for generating n x m R matrix,
# where n is the number of reads (total_reads)
# and m is each read's length (read_length).

def create_read_matrix(total_reads,read_length):
    
    read_matrix = []
    
    for i in range(total_reads):
        read_matrix.append(random_dna_sequence(read_length))
        
    return np.vstack(read_matrix)

#Build a function that simultaneously generates unique, desired length k-mers 
##from a previously generated read_matrix and times said k-mer occured.
## Returns both objects ordered by occurence

def create_kmer_matrix(read_matrix,kmer_size):
    
    kmer_matrix = []
    
    for i in range(read_matrix.shape[0]):
        
        for j in range(len(read_matrix[i])-kmer_size+1):
            kmer_matrix.append(read_matrix[i][j:j+kmer_size])
            
    kmer_matrix, counts = np.unique(kmer_matrix,axis=0,return_counts=1)
    #inner index to sort by count occurences
    sort_index = np.argsort(-counts)
    
    return kmer_matrix[sort_index], counts[sort_index]

## function to turn our kmer matrix into a one hot matrix.

def oneshotonehot(kmer_matrix):
    
    # Not yet robust, breaks when single vector is entered;
    # look into reshaping
    
    #Define bases and fit labels to use
    
    vectors = kmer_matrix[0]
    counts = kmer_matrix[1]
    oneshotonehot_matrix = []
    bases = ['A','C','T','G']
    label_encoder = LabelEncoder()
    base_encoder = label_encoder.fit(bases)
    
    for i in range(len(vectors)):
        
        encoded_base_seq = base_encoder.transform(vectors[i])
        reshaped_vector = encoded_base_seq.reshape(-1)
        one_hot_vector = np.eye(len(bases))[reshaped_vector] 
        oneshotonehot_matrix.append(one_hot_vector)
        
    return oneshotonehot_matrix, counts

## One final wrapper function to generate per-sample data on the spot

def generate_kmer_data(total_reads, read_length, kmer_size, number_of_samples):
    
    X = []
    Y = []
    sample_num = [] 
    
    for i in range(number_of_samples):
        
        read_matrix = create_read_matrix(total_reads,read_length)
        kmer_matrix = create_kmer_matrix(read_matrix,kmer_size)
        onehot_matrix = oneshotonehot(kmer_matrix)
        X.extend(onehot_matrix[0])
        Y.extend(onehot_matrix[1])
        sample_num.extend([i+1] * len(onehot_matrix[0]))
        
    return X, Y, sample_num

In [1021]:
# We will now generate and load data with our previously created helper functions
## X corresponds to one-hot encoded sequence vectors per kmer sequence
## Y corresponds to the number of times said kmer was counted
## sample corresponds to the sample identifier


## Placeholder batch size
batch_size = 64

X, y, sample = generate_kmer_data(total_reads = 100, read_length = 10 , kmer_size = 5, number_of_samples=2)

n_samples = len(X)

# We will now split our generated data into train, validation and test sets.
# With 80%, 10%, 10% of the complete dataset, respectively
## Keeping an additional sample variable with respective identifiers
### This might not be the best way to handle tags

X_train, y_train, sample_train = (X[:round(0.8*n_samples)], 
                                  y[:round(0.8*n_samples)], 
                                  sample[:round(0.8*n_samples)])
    
X_val, y_val,sample_val = (X[round(0.8*n_samples):round(0.9*n_samples)],
                           y[round(0.8*n_samples):round(0.9*n_samples)],
                           sample[round(0.8*n_samples):round(0.9*n_samples)])
        
X_test, y_test,sample_test = (X[round(0.9*n_samples):], 
                              y[round(0.9*n_samples):], 
                              sample[round(0.9*n_samples):])

In [1003]:
# We'll now create loader variables for efficient variable management.
from torch.utils.data import DataLoader, TensorDataset


train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_train),torch.FloatTensor(y_train)),
                          batch_size, shuffle=True)
validation_loader = DataLoader(TensorDataset(torch.FloatTensor(X_val),torch.FloatTensor(y_val)),
                              batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(torch.FloatTensor(X_test),torch.FloatTensor(y_test)),
                        batch_size, shuffle=False)

In [1031]:
## Pytorch NN modules
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
%matplotlib inline

## Define optimizer, RMSprop as per TTLT
## Model was trained for 200 epochs
## Learning rate = 0.001, alpha = 0.99, momentum = 0
### alpha and momentum correspond to pytorch's default value
### for the torch.optim.RMSprop class; 
### therefore, we'll only specify learning rate and epochs.

class biLSTM(nn.Module):
    
    def __init__(self, input_size, output_size, hidden_size, n_layers):
        super(biLSTM, self).__init__()
        
        self.LSTM = nn.LSTM(input_size,
                           hidden_size,
                           n_layers)
        self.linear = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        
        
        
        
        
bi_lstm = nn.LSTM(input_size=24, hidden_size = 256 , num_layers = 2, bidirectional=True)
reverse_lstm = nn.LSTM(input_size=24, hidden_size = 256 , num_layers = 2, bidirectional=True)

#We're going to make weights equal
reverse_lstm.weight_ih_l0 = bi_lstm.weight_ih_l0_reverse
reverse_lstm.weight_hh_l0 = bi_lstm.weight_hh_l0_reverse
reverse_lstm.bias_ih_l0 = bi_lstm.weight_ih_l0_reverse
reverse_lstm.bias_hh_l0 = bi_lstm.weight_hh_l0_reverse

print(bi_lstm.weight_ih_l0)
print(reverse_lstm.weight_ih_l0)
#Setting specified loss function, optimizer
criterion = nn.MSELoss()
#optimizer = optim.RMSprop(pass model parameters once built,lr=learning_rate)

n_epoch = 200
learning_rate = 0.001

# Build LSTM
## Layers = 2, 256 hidden units per layer.
## Embedding output size = 2



#bi_output, bi_hidden = bi_lstm()
#reverse_output, reverse_hidden = bi_lstm()

Parameter containing:
tensor([[ 0.0049, -0.0548,  0.0254,  ..., -0.0300, -0.0145, -0.0270],
        [-0.0332,  0.0069,  0.0244,  ..., -0.0470, -0.0490, -0.0149],
        [-0.0185, -0.0592,  0.0281,  ...,  0.0455,  0.0322,  0.0335],
        ...,
        [-0.0528,  0.0315,  0.0540,  ..., -0.0298,  0.0057, -0.0249],
        [ 0.0616, -0.0034,  0.0010,  ...,  0.0202,  0.0032,  0.0015],
        [-0.0546, -0.0334,  0.0224,  ..., -0.0281,  0.0227,  0.0377]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.0443, -0.0388,  0.0146,  ...,  0.0289, -0.0172,  0.0195],
        [ 0.0530, -0.0335,  0.0477,  ...,  0.0423,  0.0264,  0.0391],
        [-0.0485, -0.0424, -0.0608,  ...,  0.0345,  0.0427, -0.0034],
        ...,
        [ 0.0563, -0.0096,  0.0253,  ...,  0.0253,  0.0021, -0.0566],
        [-0.0510, -0.0510,  0.0578,  ...,  0.0427,  0.0557, -0.0587],
        [ 0.0189,  0.0624, -0.0231,  ..., -0.0164, -0.0038,  0.0182]],
       requires_grad=True)


In [944]:
# Build prediction layer
## MLP
## layers = 2, sizes 150 and 100, respectively
## Activation function = ReLU